# Computational Theory Assessment

## Problem 1

In [3]:
import numpy as np

def parity(x, y, z):
    """
    Parity function: x ⊕ y ⊕ z
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of XOR operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32(x ^ y ^ z)

In [4]:
def ch(x, y, z):
    """
    Choice function: (x ∧ y) ⊕ (¬x ∧ z)
    
    For each bit position, if x is 1, choose bit from y; if x is 0, choose bit from z.
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of choice operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32((x & y) ^ (~x & z))

In [ ]:
def maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    For each bit position, returns the majority bit (the bit that appears most frequently).
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of majority operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [ ]:
def rotr(x, n):
    """
    Right rotate operation for 32-bit integers
    
    Args:
        x: 32-bit integer to rotate
        n: number of positions to rotate right
        
    Returns:
        32-bit integer result of right rotation
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))

In [ ]:
def shr(x, n):
    """
    Right shift operation for 32-bit integers
    
    Args:
        x: 32-bit integer to shift
        n: number of positions to shift right
        
    Returns:
        32-bit integer result of right shift
    """
    x = np.uint32(x)
    return np.uint32(x >> n)

In [ ]:
def sigma0(x):
    """
    σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ SHR¹⁰(x)
    
    Lower case sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₀ operation
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

In [ ]:
def sigma1(x):
    """
    σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Lower case sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₁ operation
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

In [ ]:
def Sigma0(x):
    """
    Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Upper case Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₀ operation
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


In [ ]:
def Sigma1(x):
    """
    Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Upper case Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₁ operation
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


## Problem 2

## Problem 3

## Problem 4

## Problem 5